In [2]:
import pickle, gzip
from pyrosm import OSM
import osmnx as ox
import networkx as nx
import os
import platform
os.environ['USE_PYGEOS'] = '0'

In [ ]:
# system = platform.system()
# if system == "Darwin":
#     current_dir = os.path.basename(os.getcwd())
    
#     if current_dir != "backend":
#         os.chdir(os.path.join(os.getcwd(), "backend"))

In [21]:
city = 'chicago'

In [ ]:
pbf_path = "../data/osm/pbf_files/%s.osm.pbf" % city
osm = OSM(pbf_path)

In [23]:
# Get driving network with nodes=True
nodes_gdf, edges_gdf = osm.get_network(
    network_type="driving",
    nodes=True,  
    extra_attributes=["maxspeed", "lanes", "name", "oneway"]
)

In [24]:
# Build MultiDiGraph
G = nx.MultiDiGraph()

# add nodes
for _, row in nodes_gdf.iterrows():
    nid = int(row["id"])
    # OSMnx convention: x = lon, y = lat
    G.add_node(
        nid,
        x=float(row["lon"]),
        y=float(row["lat"]),
        # keep anything else if you care
        # timestamp=row["timestamp"],
        # visible=row["visible"],
    )

# add edges
for _, row in edges_gdf.iterrows():
    u = int(row["u"])
    v = int(row["v"])

    # copy all edge attributes except u,v
    data = row.to_dict()
    data.pop("u", None)
    data.pop("v", None)

    # pyrosm typically already gives 'length' (in meters) and 'geometry'
    G.add_edge(u, v, **data)

In [25]:
print("Nodes:", G.number_of_nodes(), "Edges:", G.number_of_edges())

Nodes: 637237 Edges: 741271


In [26]:
G.graph["crs"] = "EPSG:4326"

In [27]:
# add speeds & travel times via OSMnx
G = ox.add_edge_speeds(G)        # adds edge attribute 'speed_kph'
G = ox.add_edge_travel_times(G)  # adds edge attribute 'travel_time' (seconds)
# G = ox.distance.add_edge_lengths(G)   # adds 'length' attribute in meters

# convert to geodataframe
edges = ox.convert.graph_to_gdfs(G, nodes=False, edges=True)

In [28]:
edges = edges[['length', 'speed_kph', 'travel_time', 'geometry', 'width', 'name']]

In [ ]:
# Save
edges.to_feather('../data/osm/processed/%s/roads.feather' % city, compression='lz4')

In [ ]:
# Save
with gzip.open("../data/osm/processed/%s/roads.pkl.gz" % city, "wb") as f:
    pickle.dump(G, f, protocol=pickle.HIGHEST_PROTOCOL)

# Currently this newly saved file does not work for calculate_weather_route function

In [ ]:
ox.plot_graph(G, node_size=3, edge_color="gray")